<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 4*

---


# Neural Networks & hyperparameters (Prepare)
*aka Hyperparameter Tuning*

*aka Big Servers for Big Problems*

WarmUp
- [학습 규제 방식에 대한 설명 강의](https://youtu.be/_sz3KTyB9Lk?t=993)
  * L2-regularization, DropOut, Gradient Vanishing, Xavier
- [Gradient Descent With Momentum](https://youtu.be/yWQZcdJ4k8s?t=34)
  * 학습해왔던 관성의 법칙을 유지하는 방식으로 학습 개선
- [Batch Size](https://youtu.be/U4WB9p6ODjM?t=29)
  * Batch를 크게하면 좋은 이유
  * 그러나 항상 크게할 수 없는 이유
  * 일반적으로 Batch라고 하면 Mini-batch를 의미한다는 점


# 지난시간 복습
- 신경망의 순전파와 역전파 (Note 1-2)
  * 모델 생성과 모델 초기화 (Note 3)
  * 학습 과정에서 알아야 할 Tricks (Note 3)
  * 경사하강법의 다양성 (Note 2-3)
  * Overfitting 방지를 위한 노력 (Note 3)
- 그간 다뤄본 데이터
  * MNIST
  * Fashion MNIST

# 학습 목표
* <a href="#p1">Part 1</a>: 대표적인 하이퍼 파라미터를 설명 할 수 있습니다
* <a href="#p2">Part 2</a>: ETF (Experiment Tracking Framework)에 대해 알아보고 적용할 수 있습니다.
* <a href="#p3">Part 3</a>: (Optional) RandomSearch를 사용해서 하이퍼 파라미터 공간에서 최적의 하이퍼 파라미터를 찾을 수 있습니다

# Hyperparameter Options

## 개요

  하이퍼 파라미터 (Hyperparameter) 튜닝은 우리가 다뤄왔었던 다른 머신러닝 알고리즘들보다 신경망에서 훨씬 더 중요합니다. 다른 지도학습 (supervised learning) 알고리즘에는 소수의 몇몇 파라미터 정도만 손보면 됐었지만, 신경망에는 훨씬 더 많은 파라미터들을 조정해야합니다. 이 파라미터들은 모델의 정확도에 엄청난 영향을 끼치기 때문에 시간은 많이 소요되더라도 신경망을 다룬다면 반드시 거쳐야하는 단계입니다. 중요한 하이퍼파라미터들은 일전 강의에서도 몇번 다뤘습니다. 오늘은 조금 더 포괄적인 개념으로 접근해보겠습니다. 

  먼저 아셔야 할 것은 ​신경망을 위한 하이퍼 파라미터 튜닝은 쉽지않습니다. '노가다'라고 말했던 것을 기억하시나요? 만약 모델의 최종 에러 평가지표(error metric)가 Linear하지 않고 불규칙적으로 변한다면 서로 다른 하이퍼 파라미터로 지정된 모델을 어떻게 비교할 수 있을까요? 
  
  운 좋게 좋은 파라미터들이 걸릴 수도 있겠지만, 운이 안 좋아서 틀린 하이퍼 파라미터를 선택하게 되는 것을 어떻게 피할 수 있을까요? 사실 예제로 명시된 두 문제는 어떤 면에선 우리가 실험을 계속한다면 안고가야할 문제입니다. 실험들을 교차 검증(Cross Validation)한다면 최종적으로 계산되는 정확도들의 분산(variance)을 줄일 수 있어서 이러한 문제점들을 최소화가 가능합니다.

### 실습 - Boston_Housing

In [ ]:
# 데이터를 불러옵니다. 
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


### 입력 데이터 정규화 (Normalizing)

  입력 데이터를 신경망에 넣기 전에 정규화나 스케일링이 무조건 해야하는 것은 아닙니다, 보통 신경망이 수치형 데이터를 받으면 자체적으로 적절한 가중치를 학습해서 데이터를 다룰 수 있기 때문입니다. 하지만 학습을 빠르게 해주고, 경사하강법이 지역 최적점(local optimum)에 빠질 위험을 줄여주기 때문에 하는 것을 추천합니다. [읽어볼거리](https://stackoverflow.com/questions/4674623/why-do-we-have-to-normalize-the-input-for-an-artificial-neural-network)

In [ ]:
# 정규화를 위한 함수 호출
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])

[[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
   0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
  -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
   1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
  -1.30850006]
 [-0.40149354 -0.48361547 -0.86940196 -0.25683275 -0.3615597  -0.3245576
  -1.23667187  1.10717989 -0.51114231 -1.094663    0.78447637  0.44807713
  -0.65292624]
 [-0.0056343  -0.48361547  1.0283258  -0.25683275  1.32861221  0.15364225
   0.69480801 -0.57857203  1.67588577  1.5652875   0.78447637  0.3898823
   0.26349695]
 [-0.37502238 -0.48361547 -0.54747912 -0.25683275 -0.54935658 -0.78865126
   0.18954148  0.48371503 -0.51114231 -0.71552978  0.51145832  0.38669063
  -0.13812828]
 [ 0.58963463 -0.48361547

### 모델 자동 검증 기능

우리가 늘 하는 train_test_split 대신, 케라스에는 `validation_data`라는 편리한 기능이 있습니다. 모델을 학습할 때 validation_data에 테스트 데이터를 입력하면 케라스에서 자동으로 테스트셋의 일정 부분을 검증용 데이터로 사용합니다

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 중요한 하이퍼 파라미터들
inputs = x_train.shape[1]
epochs = 75                 # 전체 반복횟수
batch_size = 10             # 한번에 학습하는 사이즈


# 모델을 생성합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Sequential인 경우, 아래의 방법으로도 모델을 만들 수 있습니다.
# model = Sequential(
# [
#     Dense(64, activation='relu', input_shape=(inputs,)),
#     Dense(64, activation='relu'),
#     Dense(1)
# ]
# )

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(x_train, y_train, 
          validation_data=(x_test,y_test),  # validation set
          epochs=epochs,                    # 전체 반복횟수
          batch_size=batch_size             # 한번에 학습하는 사이즈
         )

### 하이퍼 파라미터 튜닝 방식

#### 1) Babysitting AKA "Grad Student Descent". "육아", 혹은 대학원생 갈아넣기

다윈의 진화론을 아시나요? '자연 선택'이란 단어가 진화를 주도했다고 말하는데요, 우리의 선택은 자연이 해주지 않습니다. 만약 이전의 프로젝트에서 성능을 높이기 위해서 하이퍼 파라미터를 수없이 조정해봤다면, 또는 어제 여러분이 그런 일을 해보셨다면, 바로 이것을 표현한 것입니다. 이 방법은 100% 수동으로 진행되고 있어요. 학계에서 논문을 출간할 수 있을 정도로 놀라운 정확도를 보여주는 그 하이퍼 파라미터의 수치를 찾아내기 위해 쓰는 방법이죠. 물론 지도교수님들이 이 걸 직접 하시진 않습니다, 교수님의 시간은 소중하니까요... 이 방법은 소위 노가다라고 불리고 "제출 시간 다 될 때까지 조금이라도 하이퍼 파라미터를 더 건드려본다" 방법으로도 알려져 있습니다.

#### 2) Grid Search

Grid Search는 대학원생이 "그냥 내가 실험할 것들만 컴퓨터한테 알아서 모든 조합을 돌리게 하고 난 그동안 밥이나 먹고 오면 되는 거 아닌가"라는 기가막힌 발상입니다. 하지만 Section 2에서 여러분도 이미 겪어봤겠지만 단점이 너무나 뚜렷합니다. 범위를 잘못 설정하면 여러분이 수료하고 취직을 하고나서도 끝나지 않을 가능성도 있다라는 것입니다. 가령 5개의 하이퍼 파라미터를 5개의 값으로 각각 조정하면 5^5 = 3125개의 다른 모델을 실험하게 하는 것인데 여기다 교차 검증으로 cv = 5를 하게되면 모델은 총 15,525번을 돌아야합니다. 보다시피 전수탐색 방법은 잘못 쓰면 끝나지 않는 루프속으로 빠져들지만, 잘 활용한다면 모델의 성능을 끌어올리는데 큰 도움이 됩니다.

Grid Search를 사용할 계획이라면, 여러 하이퍼 파라미터의 조합을 찾는 데 사용하지 말아야합니다. Grid Search는 오직 하나의 파라미터의 최적값을 찾는데 활용하시길 바랍니다. 다른 하이퍼 파라미터의 조합으로 모델의 성능이 획기적으로 올라가는 경우는 상당히 드뭅니다. 모델을 위한 하이퍼 파라미터만 제대로 튜닝해서 최적값을 쓰면서 하나씩 하이퍼 파라미터들을 조정해가면 못해도 90~95% 정도 원했던 성능값을 얻을 수 있습니다. 이런 식으로 접근하면 적어도 위에서 언급한 무한루프가 생길 위험을 줄일 수 있습니다.

#### 3) Random Search

Grid Search를 몇시간정도 하다보면 불현듯 자신에게 질문을 하게 됩니다. "더 나은 방법이 있을텐데..." 맞습니다, RandomSearch가 그 답입니다. RandomSearch를 활용하면 지정된 범위내에서 무작위로 선정한 조건으로 모델을 돌려본 후, 이 구성이 그나마 나으니까 이거 쓰고 이제 집에 가서 가족들이랑 시간을 보내라고 할 것입니다. 

Grid Search는 모든 파라미터가 동등하게 중요하다고 전제를 합니다, 하지만 꼭 그렇다고 할 수는 없습니다. 파라미터에 따라서 생각보다 더 크게 범위를 건너 뛰어야 되는 경우도 있습니다. Random Search는 상대적으로 중요하다고 생각되는 파라미터에 대해 탐색을 더 하고, 덜 중요한 하이퍼파라미터에 대해서는 실험을 덜 할 수 있도록 해줍니다. Random Search의 단점은 절대적으로 완벽한 하이퍼 파라미터를 찾아주진 않습니다. 하지만 Grid Search와 비교하면 덜 시간을 소모한다는 장점만으로도 충분히 빛을 발할 수 있습니다.

#### 4) Bayesian Methods

"육아" 방식과 GridSearch와 같은 수동적인 방식을 더 효과적으로 만들어 줄 수 있는 방식은 실험자가 결과를 보고 추후 탐색에 그 결과에서 얻은 정보를 반영하는 것입니다. 베이지안 최적화 방식을 활용한다면 우리의 하이퍼 파라미터 튜닝 방식의 "하이퍼 파라미터 튜닝" 할 수 있습니다. 신경망은 최적화 문제를 위한 최적화 문제 같아서 베이지안 최적화는 이전 탐색 결과를 반영해서 이후의 하이퍼 파라미터 튜닝의 성능을 높이는 전략입니다. 베이지안 방식을 활용하고 싶다면 케라스의 `keras-tuner`를 쓰면 간단하게 하실 수 있습니다. 



## 파라미터 튜닝

### 실험해볼 수 있는 하이퍼 파라미터의 종류는 다음과 같습니다:

- batch_size
- training epochs
- optimization algorithms
- learning rate (복습)
- momentum
- activation functions (복습)
- dropout regularization (복습)
- hidden layer의 neuron 갯수

더 많은 하이퍼 파라미터가 존재하지만 중요한 것들을 골라보았고 일부는 반복해서 설명하겠습니다. 

## Batch Size

배치 크기는 모델이 경사하강법을 통해 손실/오차 계산을 해서 모델의 가중치를 업데이트할 때 몇개의 관측치를 보게 되는지를 결정하는 파라미터입니다. 우리가 찾고자 하는 가장 적절한 곳은 가중치를 업데이트 할 수 있을만큼의 충분한 정보를 제공할 수 있는 딱 충분한 양의 관측치들인데, 이 때 너무 큰 배치 크기를 고르게 되면 한번에 모든 데이터에 대한 Loss를 계산해야 하는 문제점이 있고, 학습 속도가 빠르기 때문에 주어진 epoch 안에 가중치를 충분히 업데이트 할 만큼의 iteration을 돌릴 수 없게 되기에 적당한 양을 고르는 것이 중요합니다. Feed-forward 신경망은 다른 구조에 비해 배치 크기의 영향을 크게 받지 않습니다만, 그래도 튜닝이 필요한 중요한 하이퍼 파라미터입니다. 만약 너무 작은 사이즈를 고른다면 학습에 오랜 시간이 걸리고, 추정값에 노이즈가 많이 생기기 때문에 이 역시 지양해야합니다. 작은 Batchsize를 잘 고르면, Generlization이 잘 된다는 보고도 있습니다. 그러나 [BatchNorm](https://en.wikipedia.org/wiki/Batch_normalization)을 사용하는 효과를 볼 수 없다는 [보고](http://proceedings.mlr.press/v89/lian19a/lian19a.pdf)가 있습니다. 

대체적으로 배치 사이즈는 32에서 시작해서 512에서 멈추는 2의 제곱수로 결정합니다. 케라스는 default로 32를 배치사이즈로 정하고 있습니다. 얀 레컨의 유명한 트윗 중 하나를 보여드리겠습니다.
> 큰 미니배치로 학습하는 건 건강에 좋지않습니다. 주변에 혹시 32보다 큰 미니배치 쓰는 사람 있으면 돗자리 깔고 말리세요.

저 트윗의 의미가 궁금하다면 다음의 [논문](https://arxiv.org/abs/1804.07612)을 참고해보시길 바랍니다.

왜 배치 크기가 2의 제곱수로 설정되는지 궁금하다면 [다음 글](https://datascience.stackexchange.com/questions/20179/what-is-the-advantage-of-keeping-batch-size-a-power-of-2)을 읽어보시기 바랍니다



In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# 재현성을 위해 랜덤시드를 생성합니다
#seed = 7
numpy.random.seed(1100)

# 데이터셋을 불러옵니다.
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# 불러온 데이터셋을 X와 Y로 나눕니다
X = dataset[:,0:8]
Y = dataset[:,8]

# 모델을 만들기 위한 함수를 만듭니다 (KerasClassifier 요구 사항)
def create_model():
    # 모델 제작
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 모델 컴파일링
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(build_fn=create_model, verbose=0)

# GridSearch
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [30]
param_grid = dict(batch_size=batch_size)

# GridSearch CV를 만듭니다
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 최적의 결과값을 낸 파라미터를 출력합니다
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.611883544921875 using {'batch_size': 100}
Means: 0.588787031173706, Stdev: 0.09114417717420287 with: {'batch_size': 10}
Means: 0.45533486008644103, Stdev: 0.1578607973725324 with: {'batch_size': 20}
Means: 0.5313216209411621, Stdev: 0.09518651306638408 with: {'batch_size': 40}
Means: 0.5680247902870178, Stdev: 0.1355803210699447 with: {'batch_size': 60}
Means: 0.5769459366798401, Stdev: 0.0452741273316925 with: {'batch_size': 80}
Means: 0.611883544921875, Stdev: 0.05903136010705967 with: {'batch_size': 100}


## Optimizer

최적화도구(Optimizer, 옵티마이저)에 지난 시간에 많이 다뤄보았기 때문에 대해서는 간단하게 언급하고 넘어가겠습니다. 시간이 된다면 케라스에 있는 다양한 옵티마이저들에 대해 읽어보시길 바랍니다. "adam" optimizer가 보통 제일 좋은 결과를 제공합니다. 요즘은 adamW 등도 많이 사용하는데, 무엇을 선택하든 옵티마이저를 선택하는데 있어서 알아야 할 것은 옵티마이저에 따라 하이퍼 파라미터의 종류, 값 역시 달라진다는 점입니다 (learning rate, momentum, etc.) 그래서 여러분이 어떤 옵티마이저를 선택하느냐에 따라서 옵티마이저의 learning rate이나 momentum을 튜닝해야 할 수도 있습니다.

## Learning Rate

  Learning Rate은 경사하강법 기반의 optimizer 선택을 위한 하이퍼 파라미터입니다. 그러면 기존에는 어떻게 사용되었을까요? 기본 값으로 0.01로 [설정](https://www.google.com/search?q=keras+default+learning+rate&oq=keras+default+learning+rate&aqs=chrome..69i57j0i22i30l2.4191j0j7&sourceid=chrome&ie=UTF-8)되어있습니다. learning rate (학습율)이 너무 높은 경우에는 모델이 발산하게 해버리는데, 반대로 너무 낮게 설정하면 모델이 수렴하는데 실패하게 됩니다 (최적의 학습율을 찾아야 하는거죠). 일단은 크기순으로 learning rate을 튜닝합니다 ([.001, .01, .1, .2, .3, .5]). 0.5보다 높이 잡는 것은 추천하지 않지만, 한번 높이 설정한 후에 그 이유를 알아보는 것도 좋은 공부가 될 것입니다.

어느정도 범위를 좁혔다면, 갯수를 줄여서 다시 한번 해봅니다. 만약 시도한 후에 모델이 0.1을 최적의 옵티마이저라고 한다면,[.05, .08, .1, .12, .15]로 시도해서 더 좁혀들어가보면 좋을 것 같습니다 (아직 "최고"를 찾진 못했으니까요) 

학습율의 튜닝과 같이 epoch의 횟수도 튜닝하는 것이 좋습니다. 왜냐하면 learning rate이 최솟값에 도달할 수 있을 때까지 iteration의 횟수를 충분히 설정했는지를 판단할 수 있기 때문입니다.

## Momentum

모멘텀은 기본 확률적 경사 하강법과 대체적으로 많이 연관됩니다. SGD는 꽤 흔한 옵티마이저인데요, 그 이유는 사람들이 많이 이해하고 알고 있는 옵티마이저이기 때문입니다. SGD가 최고의 결과값을 도출할지는 솔직히 모르겠습니다, 하이퍼 파라미터 튜닝을 통해서 속성들을 튜닝하고 adam 옵티마이저보다 좋은 결과를 낼 수 있는지 도전해봐도 좋을 것 같습니다. 모멘텀은 옵티마이저가 최솟값을 overshooting 하게 결정하는 속성입니다. 오목한 밥그릇 한쪽에서 공을 굴리면 반대쪽에 있다가 바로 밑으로 내려가지않고 관성에 의해서 잠시 머물다 다시 내려오는 것을 상상해보세요. 모멘텀의 목적은 지역 최소점(local minima)에서 탈출하도록 시도하는 것입니다

## Activation Functions

활성화 함수에 대해선 잠깐 이야기를 나눴는데요, 보통은 은닉층에는 ReLU를 사용하고 출력층에는 Sigmoid (이진 분류)나 Softmax (다중분류)를 사용합니다, 하지만 모델에 따라서 `sigmoid`, `tanh` 등 다른 활성함수들을 시도해보고 결과가 더 괜찮게 나오는지 확인해보는 것도 좋습니다 (물론 역시는 역시나일 수도 있습니다).   

## Network Weight Initialization

우리는 처음에 망의 가중치 초기화를 어떻게 하는지에 따라 결과에 얼마나 큰 영향을 끼치는지 봤습니다. 가중치 초기화 모드는 매우 다양합니다 (아래에 종류를 적어놨습니다). 모든 것을 써볼 일은 아마 없겠지만 어떤 것을 선택하는지에 따라 모델의 초기 정확도에 큰 영향을 끼칩니다. 우리가 가중치를 잘 초기화하면 훨씬 적은 epoch으로 모델을 위한 최적의 가중치를 찾을 수 있습니다.

> 가중치 표준편차를 1인 정규분포로 초기화를 할 때 활성화 값의 분포

<img src="https://t1.daumcdn.net/cfile/tistory/994C2F3C5AB623C526" width=600/>

> 가중치의 편차를 1/sqrt(n) 으로 초기화한 Xavier 초기값의 활성화 값의 분포

<img src="https://postfiles.pstatic.net/MjAxOTA3MjlfMzgg/MDAxNTY0Mzg4Nzk5NzY4.h7xAk1g9M4giZaliyznlzL6ri5f_gdeH0v0BwUXy6ZQg.s0RUasLqzISu_NU6k2Mi-tuejq62S4AOVc_rN8pgYaMg.PNG.tinz6461/image.png?type=w773" width=580/>

> 가중치 표준편차를 sqrt(2/n)으로 초기화한 He 초기값의 활성화 값의 분포

<img src="https://postfiles.pstatic.net/MjAxOTA3MjlfMTc5/MDAxNTY0MzkyNDc3MTQx.Ge4g0BNWVB_OCwM1cbJ6QVOtIoSptaOwUrOBB4os6Awg.oOtc30idBFbagvrbyuD6m_QD80PS8_xOV39POd-n-Lsg.PNG.tinz6461/image.png?type=w773" width=580/>

c.f. Activation function에 따른 초기값 추천
   > ① Sigmoid  ⇒  Xavier 초기화를 사용하는 것이 유리 

   > ② ReLU  ⇒  He 초기화 사용하는 것이 유리


`init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']`

## 드롭아웃과 가중치 규제 (Weight Constraint)

드롭아웃 규제값은 학습 중에 무작위로 비활성화 하고 싶은 뉴런들의 비율입니다. 지난 시간에 공부했던 것처럼, 가중치 제약(Weight Constraint)은 드롭아웃과 함께 쓰는 두번째 규제 파라미터입니다. 튜닝을 할 때는 이 두개값 모두 튜닝해야합니다.
드롭아웃을 어느 layer (visible vs. hidden)에 적용하는지에 따라 다른 효과를 불러올 수 있습니다. 은닉층에 드롭아웃을 썼을 때 엄청난 효과를 부를 때도 있지만, 반대로 아무런 일도 일어나지 않을 수 있는 것 입니다. 사용하고 있는 모델이 과적합이나 일반화 문제가 있지 않다면 굳이 쓸 필요는 없습니다. 
다음 주에 공부하게될 다양한 모델들을 보시면, Dropout이 많이 적용되어 있는 것을 보실 수 있는데요, 그때에 아 여기에서는 규제방법들을 이용해서 문제를 해결해야 했구나 정도로 알고 넘어가시면 되겠습니다.

## 은닉층의 숨겨진 뉴런들


우리가 하나의 퍼셉트론만 있었을 때 선형적으로 분리가 가능한 데이터만 학습이 가능했던 것을 기억하시나요? 하지만 layer와 노드들을 신경망에 추가할 수록 비선형 데이터 학습도 가능하게 되었습니다. 신경망이 크면 클수록, 노드가 많으면 많을 수록 신경망 역시 비선형 데이터도 학습 시킬 수 있습니다. 다만 많은 노드와 레이어는 학습 시간을 늘리게 되고 모델이 과적합할 확률을 높입니다. 신경망이 커질수록 드롭아웃 규제나 다른 규제 방법으로 이러한 가능성들에 대비하고 있어야 합니다.

보통 깊이 (layer 추가)가 길이 (노드 추가)보다 신경망에 더 중요합니다. 이것이 딥러닝에 사람들이 몰리는 이유입니다. 몇가지 딥러닝 구조는 몇가지 머신러닝 프로젝트들에 혁신을 불러왔습니다.
다른 망 구조를 참조하게 될 수도 있습니다. 예를 들어 이미지 분류 문제를 풀 때 만약 resnet, inception, ResNeXt, SENet, EfficientNet 등의 SOTA (State of the Art: 최신/최고 기술)를 활용하지 않았다고 가정해보겠습니다. 그럼 아마도 좋은 성능의 모델을 찾기 위해서 수많은 시간을 실험에 할애하게 될 것입니다.
물론 휴리스틱은 존재하지만, 직접 다양한 값들을 실험해보면서 자신만의 감을 찾으면서 문제를 해결하는 것이 장기적으로 훨씬 더 많은 도움이 되기 때문에 아래의 글은 어디까지나 "참고"만 하시길 바랍니다. 
- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

# 실험 기록 프레임워크
<a id="p2"></a>

## Overview

다양한 실험을 진행하다보면 점점 결과들을 관리하는 것이 점점 힘들어지는 것을 느끼게 될 것입니다. 어떤 파라미터 조합이 제일 좋았지? 어제 했던 결과와 차이가 있었던가? 비록 파이썬 노트북을 활용하고 있지만, 사실 실험 결과를 기록하기엔 적절하지 않습니다. 이 때 Comet.ml 과 Weights and Biases가 이러한 고민들을 해결 해줄 수 있습니다.
이 도구들은 여러분이 실험을 실시간으로 기록하고, 실험에 쓰인 코드와 결과값을 보관해줍니다. 실험 결과는 원하는 평가지표로 언제든지 시각화해서 모델의 성능을 볼 수 있게 해줍니다. epoch이 끝날 때마다 데이터가 해당 툴들에 보내지고, 모델이 수렴하고 있는지 확인할 수 있습니다. 오늘은 Weights and Biases를 활용해보겠습니다

## Follow Along
 
먼저 다음 셀을 실행하기 전에 터미널에서 `wandb`에 로그인이 되있어야 합니다. 

```zsh
# 아래의 커맨드를 실행합니다
wandb.login
```
구체적인 방법은 Weights and Biases의 [QuickStart](https://docs.wandb.com/quickstart)를 참고해주시길 바랍니다

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8MB 10.2MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 133kB 37.7MB/s 
     |████████████████████████████████| 163kB 36.7MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74841 sha256=6947c64c9f265c11ef3bfbc051e30899b5ab9b59140a62ee9cde96498d099245
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=755872eb7c31849b36444f902c80936c69116a1ebdb343d20c2b385ac5eea63e
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=4fd92ca9f4293e358fdaafa80f4437bfb4af214f687d9

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
# group, project 변수를 설정합니다.
wandb_project = ""
wandb_group = ""

In [ ]:
!git clone http://github.com/wandb/tutorial

Cloning into 'tutorial'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 39 (delta 3), reused 6 (delta 2), pack-reused 31
Unpacking objects: 100% (39/39), done.


In [ ]:
!cd tutorial; pip install --upgrade -r requirements.txt;

     |████████████████████████████████| 320.4MB 47kB/s 
Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (2.4.3)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 3)) (1.1.4)
     |████████████████████████████████| 14.5MB 340kB/s 
     |████████████████████████████████| 2.2MB 41.4MB/s 
Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 6)) (0.10.11)
     |████████████████████████████████| 4.0MB 43.4MB/s 
     |████████████████████████████████| 49.5MB 70kB/s 
     |████████████████████████████████| 6.8MB 33.9MB/s 
ERROR: tensorflow 2.3.1 has requirement h5py<2.11.0,>=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
ERROR: tensorflow 2.3.1 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but yo

In [ ]:
!wandb login 6a1f7dd199ef2c241cc2dafb7ae52925d6de7385

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

wandb.init(project='dsft02') 

# 데이터 및 하이퍼파라미터 설정
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# 모델을 구축합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# 모델을 컴파일 합니다
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# 모델을 학습합니다
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

wandb: ERROR Control-C detected -- Run data was not synced


Epoch 1/50
1087/4020 [=======>......................] - ETA: 6s - loss: 8.5644 - mse: 8.5644 - mae: 2.5118

KeyboardInterrupt: ignored

In [ ]:
!ls wandb/

debug-internal.log  debug.log  latest-run  run-20201116_062105-1jbu2nbo


### Your Turn

예시에서 사용하지 않았던 하이퍼 파라미터를 몇개 골라서 튜닝을 한 후 모델을 학습해보세요. 결과는 Weights & Bias에 제출해봅니다.

In [ ]:
!wandb login 6a1f7dd199ef2c241cc2dafb7ae52925d6de7385

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
wandb_project = "n424"

wandb.init(project=wandb_project) # 초기화 및 실험 준비
wandb.config.epochs = 50  # epoch 수 설정
wandb.config.batch_size = 10 # 배치 크기 설정

# 모델을 학습합니다
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

## Challenge

오늘의 과제에서는 Weights & Biases에서 모델을 학습하고 튜닝해보는 연습을 하게 될 것 입니다

# Hyperparameters with RandomSearchCV (Learn)

## Overview

`GridSearchCV`는 시간이 너무 오래 소요됩니다. 조금 더 세련된 전략이 필요해질 것입니다.
Keras-Tuner를 사용해서 다른 방법을 찾아보겠습니다

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=6deda35942d7c6a8d987538b38c8d3d5287889544aeca43f94ba88cacfc2a81e
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=a0e740685ff28531dd1d8e8be9a6ed11717c1fba10732eaf48ebf2e22b7e39fb
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


기존 샘플로 확인해보기

In [ ]:
import pandas as pd
!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf


mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

     |████████████████████████████████| 380.5MB 43kB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 4.3MB 42.6MB/s 
     |████████████████████████████████| 501kB 44.6MB/s 


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
from kerastuner.tuners import RandomSearch

def build_model(hp):

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(units=hp.Int('units',
                                      min_value=32,
                                      max_value=256,
                                      step=32),
                                      activation='relu'),
      tf.keras.layers.Dropout(0.2), # 다음강의에서 설명이 될 component :: 존재목적 Overfitting 방지
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

    
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
#   tf.keras.layers.Dense(units=hp.Int('units',
#                                       min_value=32,
#                                       max_value=256,
#                                       step=32),
#                                       activation='relu'),
#   tf.keras.layers.Dropout(0.2), # 다음강의에서 설명이 될 component :: 존재목적 Overfitting 방지
#   tf.keras.layers.Dense(10, activation='softmax')
# ])


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='./keras-tuner-trial',
    project_name='helloworld')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2822 - accuracy: 0.9173 - val_loss: 0.1321 - val_accuracy: 0.9614
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1330 - accuracy: 0.9598 - val_loss: 0.0965 - val_accuracy: 0.9712
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1001 - accuracy: 0.9690 - val_loss: 0.0820 - val_accuracy: 0.9770
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0798 - accuracy: 0.9756 - val_loss: 0.0740 - val_accuracy: 0.9772
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0674 - accuracy: 0.9788 - val_loss: 0.0745 - val_accuracy: 0.9774
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2806 - accuracy: 0.9166 - val_loss: 0.1359 - val_accuracy: 0.9603
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1318 - accuracy: 0.9606 - val_loss: 0.0951 - val_accuracy: 0.9706

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3246 - accuracy: 0.9025 - val_loss: 0.1724 - val_accuracy: 0.9487
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2430 - accuracy: 0.9302 - val_loss: 0.1768 - val_accuracy: 0.9541
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2282 - accuracy: 0.9374 - val_loss: 0.1813 - val_accuracy: 0.9570
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2215 - accuracy: 0.9396 - val_loss: 0.1996 - val_accuracy: 0.9518
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2059 - accuracy: 0.9444 - val_loss: 0.2171 - val_accuracy: 0.9498
Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3296 - accuracy: 0.9026 - val_loss: 0.1890 - val_accuracy: 0.9448
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2535 - accuracy: 0.9281 - val_loss: 0.1555 - val_accuracy: 0.9567

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3218 - accuracy: 0.9081 - val_loss: 0.1902 - val_accuracy: 0.9483
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2422 - accuracy: 0.9343 - val_loss: 0.2003 - val_accuracy: 0.9480
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2273 - accuracy: 0.9397 - val_loss: 0.2019 - val_accuracy: 0.9536
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2218 - accuracy: 0.9436 - val_loss: 0.1814 - val_accuracy: 0.9534
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2076 - accuracy: 0.9476 - val_loss: 0.1879 - val_accuracy: 0.9586
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3131 - accuracy: 0.9070 - val_loss: 0.1890 - val_accuracy: 0.9467
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2407 - accuracy: 0.9332 - val_loss: 0.1747 - val_accuracy: 0.9506

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3366 - accuracy: 0.8977 - val_loss: 0.1833 - val_accuracy: 0.9465
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2598 - accuracy: 0.9250 - val_loss: 0.1690 - val_accuracy: 0.9532
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2374 - accuracy: 0.9312 - val_loss: 0.1581 - val_accuracy: 0.9582
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2244 - accuracy: 0.9356 - val_loss: 0.1650 - val_accuracy: 0.9604
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2215 - accuracy: 0.9373 - val_loss: 0.1909 - val_accuracy: 0.9511
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3420 - accuracy: 0.8973 - val_loss: 0.1884 - val_accuracy: 0.9442
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2550 - accuracy: 0.9257 - val_loss: 0.1761 - val_accuracy: 0.9504

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8245 - accuracy: 0.7749 - val_loss: 0.3880 - val_accuracy: 0.8995
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4057 - accuracy: 0.8857 - val_loss: 0.2958 - val_accuracy: 0.9194
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3342 - accuracy: 0.9042 - val_loss: 0.2535 - val_accuracy: 0.9296
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2931 - accuracy: 0.9161 - val_loss: 0.2269 - val_accuracy: 0.9364
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2645 - accuracy: 0.9234 - val_loss: 0.2048 - val_accuracy: 0.9416
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8469 - accuracy: 0.7683 - val_loss: 0.3846 - val_accuracy: 0.9023
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4038 - accuracy: 0.8867 - val_loss: 0.2950 - val_accuracy: 0.9189

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

# Review
* <a href="#p1">Part 1</a>: 각각에 대해서 설명할 수 있는 지, 이해되지 않은 것들은 없는지 확인해 봅시다.
    - Activation Functions
    - Optimizer
    - Number of Layers
    - Number of Neurons
    - Batch Size
    - Dropout
    - Learning Rate
    - Number of Epochs
    - and many more
* <a href="#p2">Part 2</a>: 실험 추적 프레임워크를 구현을 스스로 해볼 수 있는 지 점검합시다
    - Weights & Biases
    - Comet.ml
    - By Hand / GridSearch

## 읽어볼만한 자료
- [Grid Search Hyperparameters for Deep Learning](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)
- [Hyperparameters Optimization for Deep Learning Models](https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/)
- [Dropout Regularization in Deep Learning](https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/)
- [Weight Constraints in Deep Learning](https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/)
- [Number of Layers and Nodes in a Neural Network](https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/)
- [Batch Normalization](https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/)

